# Jupyter markdown boxes

There are two types of boxes/cells, markdown and code. The heading type is being incorporated in markdown, examples below. After entering text in markdown, you can 'run' it (button above) and it will be formatted correctly.

Handy way to make your own comments regarding code elements from a notebook.

# This is a level 1 heading
## 2 This is a level 2 heading
### level 3

Regular text in the markdown
<i>italic text</i>
<b>bold text</b>

### Jupyter and Equations:
For mathematics in the discussion and documents, we can use LaTeX markup with the modification that equations on their own line are noted by $$ before and after, rather than a \begin \end pair.

Inline equation $ Volume = \frac{4.}{3.}*\pi*r^3 $, while offset equation is: $$ Volume = \frac{4.}{3.}*\pi*r^3 $$



## Section 1: Warming up with python and random numbers

In [ ]:
#Python -- import basic math and the numpy math modules
#run this cell

from math import *
import numpy as np

### Experiment with random numbers.

Evolutionary Strategies like gaussian distributions to the parameters, with lognormal distribution to the evolution of the standard deviations

In [ ]:
#Experiment with random numbers

# Specifying the seed ensures that results are reproducible
seed = 0
np.random.seed(seed)

# Give a mean and standard deviation for a normal distribution, and for a lognormal distribution
# .. and then a number of elements to sample
mean = 0
sdev = 1.0
npts = 3200
x = np.random.normal(mean, sdev, npts)
sx = np.random.lognormal(mean, sdev, npts)

# Some demonstration output -- first element of the normal distribution, 
#    sample of the whole normal distribution and its max and min, 
#    sample of the lognormal distribution and its max and min.
# lognormal min should be >= 0.0
# Normal distribution mean should tend to zero as number of points increases
# Try making several runs with different seeds and check this
print(x[0])
print("gaussian var N(0,1) ", x, x.max(), x.min(), x.mean() )
print("lognormal var ",sx, sx.max(), sx.min() )

In [ ]:
# In any of our model or algorithm development, we will be scoring the performance in 
#    some way, often in many ways.
# Hence, there will always be some kind of scoring function, and it will return a scalar or maybe 
#    a vector/tuple/list of scores
#
# The score can be any function of the errors. RMSE is used here for demonstration purposes
#   An exponential function of the prediction error is fine, for evolution (and will
#   give very strange results -- try it)
def score(delta, start, end, tolerance = 0):
    tmp = delta[start:end]
    tmp *= tmp
    return sqrt(sum(tmp)/(end-start+1))
    #tmp = np.exp(delta[start:end])
    #return sum(tmp)/(end-start+1)
    
# Python note: the tolerance is an optional argument (and, at the moment, is not being used)
#   If it is given in the function call, the value passed will be used, otherwise it 
#   defaults to zero.
# While not used in this notebook, it will be used in the future.

### Trial of the scoring function

This will execute the scoring function on the previous block's random numbers.

As npts increases, the rmse will tend to the standard deviation. Try it.

The start and end points recognize that we will be training on a subset of the data. Train on 0-N/2, score on N/2+1 to N, for instance.

In [ ]:
start = int(0)
end = int(npts/2)
print(x[0])
print(start, end, score(x, start, end))

Quick check that the histograms look appropriate:

In [ ]:
np.histogram(x,range=(-5,5))

In [ ]:
np.histogram(sx,range=(0,10))

## Acquiring and displaying data

The test input is for a station in the MOS data set (KCGZ, in AZ, if I counted right). It includes GFS values for T2M, Td, 850-1000mb thickness, rh, and wind speed. Then t2m-observed, td-observed, t2m-error, td-error.
Once per day, for 579 days of data.

Using csv module for parsing csv file

In [ ]:
import csv
import matplotlib
import matplotlib.pyplot as plt

nobs = 579
day = np.zeros((nobs))
t2m_gfs = np.zeros((nobs))
td_gfs = np.zeros((nobs))
thick_gfs = np.zeros((nobs))
rh_gfs = np.zeros((nobs))
speed = np.zeros((nobs))
obs_t2m = np.zeros((nobs))
obs_td = np.zeros((nobs))
terr = np.zeros((nobs))
tderr = np.zeros((nobs))
#could also make up a class 'matchup', and have 579 of those

with open('testin.csv') as csvfile:
    k = 0
    sreader = csv.reader(csvfile, delimiter=",")
    for line in sreader:
        day[k] = float(line[0])
        t2m_gfs[k] = float(line[1])
        td_gfs[k] = float(line[2])
        thick_gfs[k] = float(line[3])
        rh_gfs[k] = float(line[4])
        speed[k] = float(line[5])
        obs_t2m[k] = float(line[6])
        obs_td[k] = float(line[7])
        terr[k] = float(line[8])
        tderr[k] = float(line[9])
        k += 1
csvfile.close()
        
print(day[0],t2m_gfs[0])

In [ ]:
#probably you'd rather see a graphic, so here are t2m_gfs and observed:
#matplotlib.use('Agg') #for batch mode
#matplotlib.use('Qt5Agg')

fig,ax = plt.subplots()
ax.set(xlabel="Day number", ylabel="C")
ax.plot(day,t2m_gfs,color='blue',label='GFS')
ax.plot(day,obs_t2m,color='green',label='Obs')
ax.legend()
ax.grid()
fig.show()

In [ ]:
#kluge -- my desk isn't happy with the figure show
plt.close()


fig,ax = plt.subplots()
ax.set(xlabel="Day number", ylabel="delta C")
ax.plot(day,t2m_gfs-obs_t2m)
ax.plot(day,terr)
ax.grid()
fig.show()

In [ ]:
#kluge again
plt.close()

#confirm that the difference between file's error and what we compute isn't just optically small:
print((t2m_gfs-obs_t2m-terr).max())

In [ ]:
#space for experimenting with plotting variables, say scatter plot of terr vs. the various gfs parameters:
fig,ax = plt.subplots()
ax.set(xlabel="t2m GFS", ylabel="terr")
plt.scatter(t2m_gfs, terr)
ax.grid()
fig.show()

In [ ]:
plt.close()

mean = terr.mean()
rmse = sqrt(sum(terr*terr)/nobs)
var  = sqrt(rmse*rmse-mean*mean)

print(terr.mean(), terr.mean()*1.8) #C and F mean errors
print(rmse, var, var/rmse)          #RMSE, variance

r = np.correlate(terr,t2m_gfs) / sqrt(np.correlate(terr,terr)*np.correlate(t2m_gfs,t2m_gfs)) 
print(r,r*r)


As an optical matter, it looks like there's no particular relation between how hot it is, according to the GFS, and how much the GFS is off by -- bias of almost 4 F. 

The bias being 2.15 and rms of 3.50 (rounded), suggests that simply applying a bias correction to the GFS can give substantial improvement on its own -- about 22% reduction in the rmse for removing the bias term.

The correlation (np.correlate is computing covariance, so needs the normalization) being |0.54| suggests that our eyes are largely correct -- linear regression can explain about 29% of the variance. Better than the 22% of simple bias correction, but not by much (depending, of course, on one's needs).

Of course we have more variables available, and no particular reason to believe that linear functions are the only, much less necessarily the best, ones to use.


## Preparing for the Evolutionary Strategy

It is the ambiguity about which the best variables are, what the best functions of them might be, and how best to combine them which encourages looking to evolutionary strategies -- to find improved mappings between the 5 GFS variables at hand and the one item we're trying to predict (or to correct). The principles generalize to many more input variables and many more outputs. Also, one may converge the Evolutionary Strategy towards a Neural Network. 

In [ ]:
class matchup:
    #values is a set (tuple or np.ndarray) of parameter values in the matchup
    def __init__(self, values): 
        self.values = values
    
    #display element by element the values of the matchup
    def show(self):
        n = len(self.values)
        #Note that in python for loop ranges, the iteration is over [0,n)
        for k in range(0,n):
            print(k,self.values[k])
    
    #extract the k-th parameter from the values tuple
    def __getitem__(self,k):
        return(self.values[k])
            

In [ ]:
x = matchup((0,1,2,3))
x.show()

#making a toy variable to hold a vector of matchups.
#note syntax for adding new elements.
y = []
y += [x]
y += [x]
y += [x]
print('number of elements in y: ',len(y))

  

In [ ]:
# Now bring in the data for real work:
z = []

  
with open('testin.csv') as csvfile:
    k = 0
    sreader = csv.reader(csvfile, delimiter=",")
    for line in sreader:
        day = float(line[0])
        t2m_gfs = float(line[1])
        td_gfs = float(line[2])
        thick_gfs = float(line[3])
        rh_gfs = float(line[4])
        speed = float(line[5])
        obs_t2m= float(line[6])
        obs_td = float(line[7])
        terr = float(line[8])
        tderr = float(line[9])
        
        #Note that obs_td, obs_t2m, tderr are being ignored. They can be added to the list.
        #  n.b.: note that it is terr that is used as the variable to predict, not t2m itself. 
        #Model and observation are well-enough correlated that it is the increment 
        #which makes more sense to predict [Krasnopolsky,20NNN]
        m = matchup((day,t2m_gfs,td_gfs,thick_gfs,rh_gfs,speed,terr))
        z.append(m)
        k += 1
    
csvfile.close()

#display the last-added element
print(m)
m.show()

#display it from the matchup vector
print("\n z len = ",len(z),'\n')
z[len(z)-1].show()
print("\n")

#print t2m_gfs specifically -- two different ways of referencing it. 
#The first emphasizes that z[0] is the entity, and we want the element numbered '1' from it. 
#  Remembering that indexing goes from 0 in Python.
print((z[0])[1], z[0][1])
print((z[nobs-1])[1], z[nobs-1][1])


So now z is holding our matchup set of variables. 

We now need a function whose arguments will be a matchup and a set of constants
* make a prediction about what terr should be
* return how incorrect it is

In [ ]:
#make a prediction from variables in the matchup x, using constants in the list y
#First prediction method:
def predict1(x,y):
    nx = len(x.values)
    #will ignore matchup[0] (day of observation) and matchup[6], the error term, 
    #  in making prediction
    #Starting point: simple multi-linear regression, a bias term plus 
    #  weights(y[k]) times the predictors
    pred = y[0]
    for k in range(1,nx-1):
        pred += x.values[k]*y[k]
    #print(pred, x.values[nx-1], x[nx-1])
    return (x.values[nx-1]+pred)

#take a set of matchups and evaluate (ultimately, to score) the predictions from predict1
#  note that we're now applying a start and end time -- the training period
def evaluator1(z, start, end, y):
    deltas = np.zeros((end-start+1))
    for k in range (start, end):
        deltas[k-start] = predict1(z[k],y)
    #print(deltas)
    return score(deltas,0,len(deltas))

#n.b. would be desirable to have a general evaluator that takes the 
#    prediction function as an argument as well

Very simplest, let's try the prediction being just the bias term, 2.15 C, we found above:


In [ ]:
weights = np.zeros((6))

weights[0] = 2.15
print(evaluator1(z,0,364,weights)) # score on the first year of data, where we'll do the evolving
print(evaluator1(z,365,nobs-1,weights)) #now for the remainder of the span

In [ ]:
#For comparison, try without any corrections: -- bring this forward
weights[0] = 0.0
print(evaluator1(z,0,364,weights)) 
print(evaluator1(z,365,nobs-1,weights))

### towards the evolutionary strategy

Discussion of what exactly is evolutionary will wait for next section. For now, let us continue building the framework. 

There will be a population. Each member of the population will have a set of weights (coefficients/parameter values/... -- numbers that the evaluator will make use of to make a prediction), a set of standard deviations (to specify how the weights may evolve), and a score. It is up to the evaluator, already written, to find the score, also already written. So it is the evolver that is currently needed

In [ ]:
weights = np.zeros((6))
weights[0] = 0.0

#Notice, Warning, only the first element has a nonzero standard deviation,
#  so only the first element will be nonzero when we print out the weights
sdevs = np.zeros((6))
sdevs[0] = 1.0


def evolve(weights, sdevs):
    #sdevs[0]   = np.random.lognormal(sdevs[0],0.25)
    for k in range (0,len(weights)):
        weights[k] = np.random.normal(weights[k],sdevs[k])

evolve(weights,sdevs)
print(weights)
print(sdevs)

Now to create a class to hold one of the critters (things we're going to evolve). 
Biologically speaking, these are bacteria. They evolve only by descent with mutation from a single parent, unlike diploid things, like humans, which can get genes/values from two parents.

In [ ]:
class critter:
    score = float(99.0)
    
    def __init__(self, nparm): 
        self.weights = np.zeros((nparm))
        self.sdevs = np.zeros((nparm))
    
    def init(self, weights, sdevs):
        for k in range(0,len(weights)):
          self.weights[k] = weights[k]
          self.sdevs[k]   = sdevs[k] 
            
    def copy(self, x):
        for k in range(0,len(x.weights)):
          self.weights[k] = x.weights[k]
          self.sdevs[k]   = x.sdevs[k]
        #self.show()

    #display element by element the weights and sdevs
    def show(self):
        n = len(self.weights)
#        for k in range(0,n):
        for k in range(0,1):
            print(k,self.weights[k], self.sdevs[k])
            
    def evolve(self):
        evolve(self.weights, self.sdevs)
    
    def skill(self, matchups, start, end):
        self.score = evaluator1(matchups, start, end, self.weights)


## Initial population for evolution
From evolutionary biology, the Hardy-Weinberg equilibrium, the population needs to be small enough to evolve, large enough to survive. We will cheat this some by always keeping the previous best in our next generation.

### initially, just try to re-evolve the constant bias correction


In [ ]:
npopulation = 10
population = []

nparameters = 6

for k in range (0,npopulation):
    population.append(critter(nparameters))
    
population[npopulation-1].show()

sdevs[0] = 1.0
for k in range (0,npopulation):
    weights[0] = np.random.normal(0,1)
    population[k].init(weights,sdevs)
    print(k, population[k].weights[0], population[0].weights[0] )

population[0].show()
population[1].show()

#How to copy element
population[1].copy(population[0])
population[1].show()

In [ ]:
#recall that the poorly-named 'z' is holding the matchups
smin = 9999.
kbest = int(99)

for k in range (0,npopulation):
    population[k].skill(z, 0, 364)
    if (population[k].score < smin):
        kbest = k
        smin = population[k].score 
    
print("k, smin = ",kbest, smin)
population[kbest].show()

In [ ]:
#swap best in to all slots
#then evolve a new population of critters from that
#evaluate them
#repeat until limit of generations or you are satisfied
generation_max = int(300)
population[kbest].show()

#Not necessary to redefine, but gives somewhat better results for 
population[0].sdevs[0] = 0.25

#So many simplifications have been made so already that little difference between 
#  (population,generation limit) being (10,300) vs. (3000,1)
#Our demonstrations will soon become more complex and the difference will be very large

for gen in range(0,generation_max):
    population[0].copy(population[kbest])
    population[0].score = population[kbest].score
    score_best = float(population[0].score)
    smin = score_best
    kbest = 0
    #Try npop times, then evolve from the best of these
    for k in range (1, npopulation):
        population[k].copy(population[0])
        population[k].evolve()
        population[k].skill(z,0,364)
        #Sense of score is that smaller is better
        if (population[k].score < score_best):
            kbest = k
            smin = population[k].score
    if (kbest != 0):
        print("new best ",gen, kbest, smin, score_best)
        population[kbest].show()
        
population[kbest].show()

print("\n")
print(evaluator1(z, 365,nobs,population[kbest].weights))
